In [161]:
# June 20 2018
# script to predict titanic kaggle competion survival rates
# author - Benson
# language - python 2

# load the libraries needed
import numpy as np # scientific computing library
import pandas as pd # library for data analysis

from sklearn.ensemble import RandomForestClassifier #  ML classier algorithm
import random # to randomly replace missing values

In [162]:
# load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# explore the entries
train.head(2)
# test.head()
#train.info()
#test.info()

# In summary 
# 891 rows in train dataset
# 418 rows in test dataset
# Trainn - All values captured: PassengerId, Survived, Pclass, Sex, SibSp, Parch, Ticket, Fare
# Missing values: Age, Cabin, Embarked

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [163]:
# feature engineering
# ----------------------

# My 4 variables of interest
# Embarked, Age, PClass, Sex, 

# drop unneeded columns - train
trimmed_train=train.drop(['PassengerId','Name','SibSp','Parch','Ticket', 'Fare', 'Cabin'], axis=1)

# drop in test dataframe
trimmed_test=test.drop(['Name','SibSp','Parch','Ticket', 'Fare', 'Cabin'], axis=1)


In [164]:
# 1. Embarked
# get unique values of Embarked and their distribution
# print trimmed_train['Embarked'].unique()
# print trimmed_test['Embarked'].describe()

# replace nan with a random str from the other 3 choices
trimmed_train['Embarked']=trimmed_train['Embarked'].fillna(random.choice(['S', 'Q','C']))
trimmed_test['Embarked']=trimmed_test['Embarked'].fillna(random.choice(['S', 'Q','C']))

# convert to int 1-S 1-Q 2-C
trimmed_train['Embarked'] = trimmed_train['Embarked'].map({'S': 0, 'Q': 1, 'C': 2})
trimmed_test['Embarked'] = trimmed_test['Embarked'].map({'S': 0, 'Q': 1, 'C': 2})

# print trimmed_train.info()
# print trimmed_test.head(20)

# Embarked sorted


In [165]:
# 2 Age
# check if missing in boths datasets

# trimmed_train.info()
# trimmed_test.info()

# get mean ages
train_mean_age = trimmed_train['Age'].mean()
test_mean_age = trimmed_test['Age'].mean()

# Standard deviation
train_sd_age = trimmed_train['Age'].std()
test_sd_age = trimmed_test['Age'].std()

# lower and upper bounds
train_lower_bound = train_mean_age - train_sd_age
train_upper_bound = train_mean_age + train_sd_age

test_lower_bound = test_mean_age - test_sd_age
test_upper_bound = test_mean_age + test_sd_age

# generate random ages within bounds
train_random_age =  random.randint(int(train_lower_bound), int(train_upper_bound))
test_random_age =  random.randint(int(test_lower_bound), int(test_upper_bound))

# convert Age to int and drop na values
trimmed_train['Age'] = trimmed_train['Age'].dropna().astype(int)
trimmed_test['Age'] = trimmed_test['Age'].dropna().astype(int)

# replace nan with random ages
trimmed_train['Age']=trimmed_train['Age'].fillna(train_random_age)
trimmed_test['Age']=trimmed_test['Age'].fillna(test_random_age)

# trimmed_test.head(20)
# trimmed_train.info()

# Age sorted

In [166]:
# 3 PClass
# trimmed_test.info()
# No missing values
trimmed_train.head(5)

# PClass sorted

,Survived,Pclass,Sex,Age,Embarked
0,0,3,male,22.0,0
1,1,1,female,38.0,2
2,1,3,female,26.0,0
3,1,1,female,35.0,0
4,0,3,male,35.0,0


In [176]:
# 4 Sex
# trimmed_test.info()
# trimmed_train.info()
# trimmed_train['Sex'].unique()
# trimmed_test['Sex'].unique()

# convert to int 0 female 1 male
trimmed_train['Sex']=trimmed_train['Sex'].map({'female':0, 'male': 1})
trimmed_test['Sex']=trimmed_test['Sex'].map({'female':0, 'male': 1})
print trimmed_test.head(5)
# Sex sorted


   PassengerId  Pclass  Sex   Age  Embarked
0          892       3  NaN  34.0         1
1          893       3  NaN  47.0         0
2          894       2  NaN  62.0         1
3          895       3  NaN  27.0         0
4          896       3  NaN  22.0         0


In [175]:
# training and testing set
trimmed_train.head()
X_train = trimmed_train.drop("Survived",axis=1)
Y_train = trimmed_train["Survived"]
X_test  = trimmed_test.drop("PassengerId",axis=1).copy()

# random forest classifier
rf=RandomForestClassifier(n_estimators=100)

# train
rf.fit(X_train, Y_train)

# predict
Y_predict=rf.predict(X_test)

# score
rf.score(X_train, Y_train)


0.89898989898989901

In [174]:
# finally save my prediction
prediction = pd.DataFrame({'PassengerId': trimmed_test['PassengerId'], 'Survived': Y_predict})

# save to csv
prediction.to_csv('prediction.csv', index=False)

# show me some outputs
prediction.head(10)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,1
5,897,0
6,898,0
7,899,0
8,900,0
9,901,0


In [173]:
print 'Hurrayy!!!'

Hurrayy!!!
